In [ ]:
import tarfile
from pathlib import Path


def extract_cli_archive():
    src = "archive.tar.gz"
    dst = Path("/tmp/cli_archive")

    with tarfile.open(src, "r:gz") as tar:
        tar.extractall(path=dst)

    print(f"Extracted {src} to {dst}")


extract_cli_archive()

In [ ]:
import os
import json
from dbruntime.databricks_repl_context import get_context
import subprocess
import multiprocessing


class Runner:
    def __init__(self, archive_dir):
        # Load environment variables to set in the test runner.
        self.env = os.environ.copy()
        with open("params.json", "r") as f:
            params = json.load(f)
        for k, v in params.items():
            self.env[k] = v

        # Configure PATH to include go, uv and jq.
        self.env["PATH"] = f"{archive_dir}/bin/amd64:{archive_dir}/bin/amd64/go/bin:{self.env['PATH']}"

        ctx = get_context()
        workspace_url = spark.conf.get("spark.databricks.workspaceUrl")

        # Configure auth for the acceptance tests.
        self.env["DATABRICKS_TOKEN"] = ctx.apiToken
        self.env["DATABRICKS_HOST"] = workspace_url

        # Configure working directory to the root of the CLI repo.
        self.exec_dir = Path(archive_dir) / "cli"

        # Terraform needs to be installed on the workspace file system to be used by the acceptance tests.
        # Otherwise creating symlinks in .databricks fails.
        self.terraform_dir = os.getcwd() + "/terraform"

    def run(self, prefix):
        cmd = [
            "go",
            "tool",
            "gotestsum",
            "--format",
            "testname",
            "--no-summary=skipped",
            "--",
            "-timeout",
            "7200s",
            "-test.v",
            "-run",
            prefix,
            "github.com/databricks/cli/acceptance",
            "-workspace-tmp-dir",
            "-terraform-dir",
            self.terraform_dir,
            "-tail",
        ]

        subprocess.run(cmd, env=self.env, check=True, cwd=self.exec_dir)

    # Debug helper to run commands and see the output.
    def run_script_with_logs(self, name, input_script):
        cmd = [
            "go",
            "tool",
            "gotestsum",
            "--format",
            "testname",
            "--no-summary=skipped",
            "--",
            "-timeout",
            "7200s",
            "-test.v",
            "-run",
            "TestAccept/" + name,
            "github.com/databricks/cli/acceptance",
            "-workspace-tmp-dir",
            "-terraform-dir",
            self.terraform_dir,
            "-tail",
        ]

        with open(self.exec_dir / "acceptance" / name / "script", "w") as f:
            f.write(input_script)

        subprocess.run(cmd, env=self.env, check=True, cwd=self.exec_dir)

    # Debug helper to run commands and see the output.
    def run_with_script(self, name, input_script):
        cmd = [
            "go",
            "test",
            "-timeout",
            "7200s",
            "-test.v",
            "-run",
            "TestAccept/" + name,
            "github.com/databricks/cli/acceptance",
            "-update",
            "-workspace-tmp-dir",
            "-terraform-dir",
            self.terraform_dir,
        ]

        with open(self.exec_dir / "acceptance" / name / "script", "w") as f:
            f.write(input_script)

        try:
            subprocess.run(
                cmd, env=self.env, check=True, cwd=self.exec_dir, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL
            )
        except subprocess.CalledProcessError:
            pass

        with open(self.exec_dir / "acceptance" / name / "output.txt", "r") as f:
            print(f.read())

: 

In [ ]:
runner = Runner("/tmp/cli_archive")
runner.run_with_script(
    "selftest/dbr",
    r"""
echo "========================= START OF SCRIPT ====================="
DASHBOARD_DISPLAY_NAME="test bundle-deploy-dashboard $(uuid)"
export DASHBOARD_DISPLAY_NAME

trace cat databricks.yml

envsubst < databricks.yml.tmpl > databricks.yml

trace cat databricks.yml
echo "========================= END OF SCRIPT ====================="
""",
)

In [ ]:
# Call .run on the runner to run the tests. Please the the previous cells before running tests here.

# Example:
runner = Runner("/tmp/cli_archive")
runner.run("selftest/dbr")